In [63]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
#import tensorflow as tf
import torch
from torch_geometric.data import Data
from urllib.parse import urlparse


In [50]:
def create_dict(table, col):
    table_dict = {}
    for ix, value in enumerate(table[col]):
        #print(value)
        if value != value:
            continue
        if value in table_dict:
            table_dict[value].append(ix)
        else:
            table_dict[value] = [ix]
    return table_dict

def search_table(table, column):
    connections = []
    lookup_dict = create_dict(table, column)
    for ix, value in enumerate(table[column]):
        if value in lookup_dict:
            for item in lookup_dict[value]:
                connections.append((ix, item, column, value))
    connections = pd.DataFrame(
        connections, 
        columns=['source', 'target', 'column', 'value']
    )
    return connections

## Load Data

In [54]:
zync = pd.read_csv("data/zync_session_tracking_orc_20210625_all.csv")
print(f"Zync data shape: {zync.shape}")

sizmek = pd.read_csv("data/sizmek_bidstream_raw_20210625_10k.csv")
print(f"Sizmek data shape: {sizmek.shape}")

# Fix column headers
zync.columns = [i.split(".")[1] for i in zync.columns]
sizmek.columns = [i.split(".")[1] for i in sizmek.columns]

def make_smaller(df, frac):
    new_df = df.copy()
    new_df = new_df.sample(frac=frac, replace=False)
    return new_df

#zync = make_smaller(zync, 0.1)
#sizmek = make_smaller(sizmek, 0.1)

Zync data shape: (99999, 16)
Sizmek data shape: (10000, 16)


In [55]:
sizmek.head(1)

,user_id,household_cluster_id,zeta_user_id,account_id,url,referrer_url,city_code,state_code,zip_code,dma_code,country_code,user_agent,mobile_id,ip,server_timestamp,dt
0,163543465731194,NaN,NaN,19967,http://mob.app.de.wetteronline.wetterapp,NaN,3103824.0,NaN,NaN,NaN,PL,Mozilla/5.0 (Linux; Android 11; SM-A715F Build...,b7f4d610-7879-4394-aa2b-bfb9f2b1c25b,31.0.26.0,1624622086664,20210625


In [56]:
zync.head(1)

,session_id,client,client_id,remote_addr,referrer,user_agent_platform,user_agent_language,user_agent_browser,user_agent_string,user_agent_version,timestamp,geo_data,partner_data,session,consent_data,dt
0,042daa1a-aa73-47f0-89f4-5ead6821767e:161417230...,disqus,c3rj9u181f91uc2,128.201.40.29,https://www.biorxiv.org/content/10.1101/560755...,windows,NaN,chrome,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,91.0.4472.114,1.624644e+09,"{""country"":""BR"",""subdivision"":""RJ"",""coordinate...",NaN,"{""clients"":{""appnexus"":null,""disqus"":""c3rj9u18...","{""gdpr"":null,""gdpr_consent"":null}",2021-06-25-18


## Transform Data

#### Create Edges
Parse the base url domain for increased matching and then apply the functions defined up top to create the list of edge connections within the dataset.

In [57]:
sizmek["url_domain"] = sizmek['url'].apply(
    lambda x: urlparse(x).netloc if pd.notnull(x) else x
)
sizmek_edges = search_table(sizmek, "url")
sizmek_edges.head(2)

In [60]:
sizmek['url_domain'].value_counts()[:5]

centurylink.net                      3354
                                     2871
mob.app.G15147002586                  589
abolt.newsweek.com                    226
mob.app.com.abtnprojects.ambatana     214
Name: url_domain, dtype: int64

#### Create Features
Use a standard one-hot encoding method of select columns and transform to a new DataFrame

In [65]:
feature_cols = ["account_id", "referrer_url", "city_code", 
                "state_code", "dma_code", "country_code"]

feature_enc = OneHotEncoder(handle_unknown="ignore")
sizmek_features = pd.DataFrame(
    feature_enc.fit_transform(sizmek[feature_cols]).toarray(), 
    columns=feature_enc.get_feature_names(feature_cols)
)
print(f"Unique features: {sizmek_features.shape[1]}")

label_enc = LabelEncoder()
labels = label_enc.fit_transform(sizmek["zip_code"])
print(f"Unique labels: {max(labels)}")

Unique features: 596
Unique labels: 312


#### Create Graph
Use PyTorch Geometric to create a graph with their standard graph class `torch_geometric.data.Data`

In [ ]:
sizmek_graph = Data(
    x=torch.tensor(sizmek_features.values, dtype=torch.float),
    edge_index=torch.tensor(sizmek_edges[['source','target']].values, dtype=torch.long)
)

## Model